In [29]:
import os
from pydub import AudioSegment
import whisper
from datetime import timedelta
import math

In [30]:
def chunk_audio(file_path, chunk_length_ms=60000):
    audio = AudioSegment.from_file(file_path, format="mp3")
    length_audio = len(audio)
    chunks = []
    for i in range(0, length_audio, chunk_length_ms):
        chunks.append(audio[i:i+chunk_length_ms])
    return chunks

def format_time(seconds):
    rounded_seconds = math.ceil(seconds)  # Round up the seconds
    return str(timedelta(seconds=rounded_seconds))

def process_transcription(result):
    segments = result['segments']
    formatted_sentences = []
    for segment in segments:
        start = format_time(segment['start'])
        text = segment['text']
        formatted_sentences.append(f'[{start}] {text}')
    return formatted_sentences

def transcribe_chunk(chunk, model, elapsed_seconds):
    chunk.export("temp_chunk.mp3", format="mp3")
    result = model.transcribe("temp_chunk.mp3")
    os.remove("temp_chunk.mp3")

    # Adjust segment times based on the elapsed time
    for segment in result['segments']:
        segment['start'] += elapsed_seconds
        segment['end'] += elapsed_seconds

    # Update the elapsed time for the next chunk
    last_segment = result['segments'][-1] if result['segments'] else {'end': 0}
    new_elapsed_seconds = last_segment['end']

    return result, new_elapsed_seconds

def transcribe_audio(file_path, chunk_length_ms=60000):
    model = whisper.load_model("base")
    audio_chunks = chunk_audio(file_path, chunk_length_ms)
    directory = os.path.dirname(file_path)
    output_file = os.path.join(directory, "transcription_output.txt")
    
    elapsed_seconds = 0  # Initialize elapsed time
    with open(output_file, 'w') as f:
        for i, chunk in enumerate(audio_chunks):
            try:
                print(f"Transcribing chunk {i+1}/{len(audio_chunks)}...")
                result, elapsed_seconds = transcribe_chunk(chunk, model, elapsed_seconds)
                sentences = process_transcription(result)
                for sentence in sentences:
                    f.write(sentence + '\n\n')
            except Exception as e:
                print(f"Error transcribing chunk {i+1}: {e}")

In [31]:
path_to_mp3 = "/home/jason/Desktop/Study/DR/SampleAudio/TED.mp3"
transcribe_audio(path_to_mp3)

Transcribing chunk 1/9...
Transcribing chunk 2/9...
Transcribing chunk 3/9...
Transcribing chunk 4/9...
Transcribing chunk 5/9...
Transcribing chunk 6/9...
Transcribing chunk 7/9...
Transcribing chunk 8/9...
Transcribing chunk 9/9...
